### Design AB testing

Выявление особенностей данных. Здесь наша основная цель — составить общее представление о данных. Нет точного алгоритма, который покажет вам все нужные метрики по данным, но попробуйте отталкиваться от того, что вы хотите сделать дальше. Если вы работаете с уже знакомыми данными, то, скорее всего, для вас этот пункт не особо важен. Если же нет, можете опираться на следующие базовые вопросы:

- За какой период представлены данные?
- Какое уникальное количество каждой сущности? Какие у них распределения?
- Если мы имеем дело с транзакциями, то какое в среднем количество покупок с каким средним чеком совершают пользователи за некоторый период времени?
- если имеем дело с мобильными приложениями и подписками, то какой у них retention и с каким чеком?

Сбор необходимых метрик. Обычно мы работаем с гипотезами, которые растят какую-то часть нашей воронки, поэтому на данном этапе мы будем её восстанавливать в разных разрезах/сегментах/когортах.

Оценка возможного влияния. На этом этапе наша задача состоит в том, чтобы оценить сверху и снизу, насколько предполагаемое решение может изменить ключевую метрику. Тут можно брать значения, которые точно не вписываются в адекватную картину мира :)

Подведение итогов. Здесь мы принимаем решение о том, стоит ли точно принять/отвергнуть придуманную гипотезу, либо однозначного ответа не было найдено, и надо проводить А/B-тест.

### Введение 

На главной странице магазина пользователям показываются баннеры, чтобы стимулировать продажи. Сейчас там отображается один из 5 возможных баннеров случайным образом. Каждый баннер рекламирует определённый продукт или всю компанию. Маркетологи магазина считают, что опыт работы с баннерами может варьироваться в зависимости от сегмента, а их эффективность может зависеть от характеристик поведения пользователя.

Менеджер компании получил предложение от партнёров продать это место под баннер и рекламировать там другую компанию и их услуги. Оплата принимается по модели CPC (cost per click, то есть фиксируется цена за каждый клик).

Помогите менеджеру принять верное решение на основе данных.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('ab_design.csv')

### Описание данных

- order_id — номер заказа
- user_id — уникальный идентификатор покупателя
- page_id — идентификатор страницы, где было совершено действие
- product — что показывалось на баннере
- site_version — версия сайта: мобильная/десктопная
- time — время действия
- title — вид действия (клик/показ/покупка)
- target — была ли совершена покупка

In [3]:
df

,order_id,user_id,page_id,product,site_version,time,title,target
0,cfcd208495d565ef66e7dff9f98764da,c81e728d9d4c2f636f067f89cc14862c,6f4922f45568161a8cdf4ad2299f6d23,sneakers,desktop,2019-01-11 09:24:43,banner_click,0
1,c4ca4238a0b923820dcc509a6f75849b,eccbc87e4b5ce2fe28308fd9f2a7baf3,4e732ced3463d06de0ca9a15b6153677,sneakers,desktop,2019-01-09 09:38:51,banner_show,0
2,c81e728d9d4c2f636f067f89cc14862c,eccbc87e4b5ce2fe28308fd9f2a7baf3,5c45a86277b8bf17bff6011be5cfb1b9,sports_nutrition,desktop,2019-01-09 09:12:45,banner_show,0
3,eccbc87e4b5ce2fe28308fd9f2a7baf3,eccbc87e4b5ce2fe28308fd9f2a7baf3,fb339ad311d50a229e497085aad219c7,company,desktop,2019-01-03 08:58:18,banner_show,0
4,a87ff679a2f3e71d9181a67b7542122c,eccbc87e4b5ce2fe28308fd9f2a7baf3,fb339ad311d50a229e497085aad219c7,company,desktop,2019-01-03 08:59:15,banner_click,0
...,...,...,...,...,...,...,...,...
8471215,70c275428b8d53eef294d0529253b694,59e736f90b5f8003072bf0eb271ddb86,7bc3a33568d00773d5b58d6c7348bf3e,accessories,desktop,2019-05-23 14:07:00,banner_show,0
8471216,94c9a2a0c11c2b88e49c13e401608c86,69d73bfcaaa83217c903627e602ae01c,f236830bcbb2d279c0c7c3d7f5d3a970,clothes,mobile,2019-05-28 08:10:20,banner_show,0
8471217,862568e6013bc378b06f43c3e466521a,61728d26a9c9feb9e128c17b400062c8,4fc14437e418f19200e71acf4f34b3ba,sports_nutrition,mobile,2019-05-20 09:20:50,banner_show,0
8471218,7c70b42806372ba85f47a3956f23a114,f90e8c3bcf3867e65d62a2a08189a35a,c6a3e7758b9c0a43417c4a2c1a4504dd,sneakers,mobile,2019-05-28 19:25:42,banner_show,0


### Выявление особенностей в данных

Давайте посмотрим на базовые статистик по таким данным, чтобы лучше понять, как с ними работать.

In [4]:
# Посмотрим на минимальное и максимально время
df.time.min(), df.time.max()

('2019-01-01 00:00:03', '2019-05-31 23:59:58')

In [5]:
# Посмотрим на размер данных
df.shape

(8471220, 8)

In [6]:
# Посмотрим на кол-во уникальных пользователей
df.user_id.nunique()

4254619

In [7]:
# Посмотрим сколько в среднем устройств приходиться на пользователя
df.groupby('user_id').site_version.nunique().mean()

1.0064259572948835

In [8]:
# Посмотрим на среднее кол-во чеков среди тех кто что-то купил
df[df.title == 'order'].groupby('user_id').order_id.nunique().mean()

1.0456391413653066

### Сбор необходимых метрик

У нас есть гипотеза, основанная на предположениях что конверсия в покупку будет отличаться в зависимиости от того, с какого девайся заходит пользователь, и от того, какой баннер ему показываеться. Давайте это проверим, восстановив всю воронку с баннерами на сайте.

In [9]:
new_df = {
    'site_version': [],
    'banner_type': [],
    'num_shows': [],
    'num_clicks': [],
    'num_orders': [],
    'conversion_show_click': [],
    'conversion_click_order': [],
    'conversion_show_order': []
}

for site_version, group in df.groupby('site_version'):
    for prod, pgroup, in group.groupby('product'):
        num_shows = pgroup[pgroup['title'] == 'banner_show'].shape[0]
        num_clicks = pgroup[pgroup['title'] == 'banner_click'].shape[0]
        num_orders = pgroup[pgroup['title'] == 'order'].shape[0]

        new_df['site_version'].append(site_version)
        new_df['banner_type'].append(prod)
        new_df['num_shows'].append(num_shows)
        new_df['num_clicks'].append(num_clicks)
        new_df['num_orders'].append(num_orders)
        new_df['conversion_show_click'].append(num_clicks / num_shows)
        new_df['conversion_click_order'].append(num_orders / num_clicks)
        new_df['conversion_show_order'].append(num_orders / num_shows)

Переводим данные в датафрейм и посмотрим на получившуюся таблицу



In [10]:
new_df = pd.DataFrame(new_df)
new_df

,site_version,banner_type,num_shows,num_clicks,num_orders,conversion_show_click,conversion_click_order,conversion_show_order
0,desktop,accessories,410003,18531,22121,0.045197,1.193729,0.053953
1,desktop,clothes,418070,32781,66977,0.078410,2.043165,0.160205
2,desktop,company,477374,28464,0,0.059626,0.000000,0.000000
3,desktop,sneakers,411597,21419,32565,0.052039,1.520379,0.079119
4,desktop,sports_nutrition,417595,13870,11518,0.033214,0.830425,0.027582
5,mobile,accessories,1030951,117723,22430,0.114189,0.190532,0.021757
6,mobile,clothes,1035058,187814,45738,0.181453,0.243528,0.044189
7,mobile,company,1102861,116357,0,0.105505,0.000000,0.000000
8,mobile,sneakers,1041430,161177,35154,0.154765,0.218108,0.033756
9,mobile,sports_nutrition,1048375,131048,12219,0.125001,0.093241,0.011655


### Оценка возможного влияния

В силу того, что конверсии между собой отличаются, рекомендуется производить А/B-тестирование на все 10 возможных вариантов. И, если разработка и руководство допускают такой вариант, предложить заменить только некоторые — те, которые принесут больше денег.

Но сначала надо посчитать потенциальную выгоду в деньгах: возможно, мы можем дать точный ответ и без тестирования.

Обычно о цене за клик договариваются напрямую с заказчиком, поэтому для нас это будет просто очередной вводной для дизайна эксперимента. Давайте смоделируем наш дизайн со следующими входными данными:

- Средняя цена товара — 300 рублей
- Цена за клик — 20 рублей
- Покупательский цикл — 7 дней

In [11]:
cpc = 1
avg_prod_price = 1000

In [12]:
new_df['cpc_now'] = new_df['num_orders'] * avg_prod_price / new_df['num_clicks']
new_df['min_needable_cr'] = ((new_df['num_orders'] * avg_prod_price) / cpc) / new_df['num_shows']

Здесь мы смоделировали количество денег, которые приносит каждый клик сейчас, и рассчитали необходимую конверсию на новые баннеры, чтобы их обновление окупилось.

Посмотрим на полученные значения:

In [13]:
new_df[['site_version', 'banner_type', 'cpc_now', 'min_needable_cr']]

,site_version,banner_type,cpc_now,min_needable_cr
0,desktop,accessories,1193.729426,53.953264
1,desktop,clothes,2043.165248,160.205229
2,desktop,company,0.000000,0.000000
3,desktop,sneakers,1520.379103,79.118652
4,desktop,sports_nutrition,830.425379,27.581748
5,mobile,accessories,190.532012,21.756611
6,mobile,clothes,243.528171,44.188828
7,mobile,company,0.000000,0.000000
8,mobile,sneakers,218.108043,33.755509
9,mobile,sports_nutrition,93.240645,11.655181


Как мы видим, при текущей конверсии смоделированная выручка за клик почти во всех вариантах является больше. То есть предварительно можно было бы сразу отвергнуть предложение обновить баннеры при такой средней цене товара и цене за клик, потому что вряд ли мы получим конверсию выше, чем в данный момент.

Более того, если бы наши вводные были следующими:

- Средняя цена товара — 1000 рублей,
- Цена за клик — 1 рубль,

то необходимые значения конверсий выглядели бы так: